In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [4]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [5]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
# inserting a column - class as target feature
df_fake["class"] = 0
df_true["class"] = 1

In [8]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [12]:
# Step 1: Separate last 10 rows from each DataFrame
df_fake_manual_testing = df_fake.tail(10).copy()
df_true_manual_testing = df_true.tail(10).copy()

# Step 2: Remove the last 10 rows from each original DataFrame
df_fake = df_fake.iloc[:-10]
df_true = df_true.iloc[:-10]

# Step 3: Add class labels
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

df_fake.shape, df_true.shape

((23441, 5), (21377, 5))

In [17]:
df_fake_manual_testing.head(5)

,title,text,subject,date,class
23441,Wikileaks: NSA Spied on UN Secretary-General a...,InverseWikileaks released tonight a new cache...,Middle-east,"February 23, 2016",0
23442,"SHOUT POLL: Will Donald Trump hold his lead, o...",21st Century Wire asks HAVE YOUR SHOUT: With ...,Middle-east,"February 21, 2016",0
23443,ENCRYPTION TRUTH: What The FBI Aren’t Telling ...,Shawn Helton 21st Century WireEarlier this wee...,Middle-east,"February 20, 2016",0
23444,"BOILER ROOM – EP #44 – Dig, Dug, Dirt!",Tune in to the Alternate Current Radio Network...,Middle-east,"February 19, 2016",0
23445,GLOSSED OVER: Key Questions Emerge After Death...,21st Century Wire asks What really happened in...,Middle-east,"February 17, 2016",0


In [18]:
df_true_manual_testing.head(5)

,title,text,subject,date,class
21377,Barcelona balances security and freedom after ...,MADRID (Reuters) - Spain s northeastern region...,worldnews,"August 23, 2017",1
21378,Cambodia accuses U.S. of political interferenc...,PHNOM PENH (Reuters) - Cambodia hit back on Th...,worldnews,"August 24, 2017",1
21379,Australian government faces uncertain two mont...,SYDNEY (Reuters) - A citizenship crisis will l...,worldnews,"August 24, 2017",1
21380,Vietnam calls for Southeast Asian unity amid S...,HANOI (Reuters) - Vietnam s most powerful lead...,worldnews,"August 24, 2017",1
21381,Canada frets over possible huge surge in asylu...,OTTAWA (Reuters) - Canada fears a huge surge i...,worldnews,"August 23, 2017",1


In [16]:
# merging true and fake dataframes
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(5)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [19]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [20]:
# removing colums which are not required
df = df_merge.drop(["title", "subject","date"], axis = 1)
df.isnull().sum()

,0
text,0
class,0


In [21]:
#random shuffling the dataframes
df = df.sample(frac = 1)
df.head()

,text,class
20433,This is what happens when people are more conc...,0
1226,WASHINGTON (Reuters) - Secretary of State Rex ...,1
2641,NEW YORK (Reuters) - There is at least one app...,1
12771,DUBLIN (Reuters) - The Brexit transition perio...,1
14000,What kind of monsters teach their children to ...,0


In [23]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.columns

Index(['text', 'class'], dtype='object')

In [24]:
df.head()

,text,class
0,This is what happens when people are more conc...,0
1,WASHINGTON (Reuters) - Secretary of State Rex ...,1
2,NEW YORK (Reuters) - There is at least one app...,1
3,DUBLIN (Reuters) - The Brexit transition perio...,1
4,What kind of monsters teach their children to ...,0


In [25]:
# creating a function to process the text
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df["text"] = df["text"].apply(wordopt)

In [28]:
# defining dependent and independent variables
x = df["text"]
y = df["class"]

In [30]:
# splitting training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [34]:
#logistic regression
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [35]:
pred_lr=LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9883980365908077

In [36]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5843
           1       0.99      0.99      0.99      5362

    accuracy                           0.99     11205
   macro avg       0.99      0.99      0.99     11205
weighted avg       0.99      0.99      0.99     11205



In [37]:
# decision tress classifier
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [38]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9966979027219991

In [39]:
print (classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5843
           1       1.00      1.00      1.00      5362

    accuracy                           1.00     11205
   macro avg       1.00      1.00      1.00     11205
weighted avg       1.00      1.00      1.00     11205



In [42]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)


RandomForestClassifier(random_state=0)

In [43]:
pred_rfc = RFC.predict(xv_test)
RFC.score(xv_test, y_test)

0.9906291834002677

In [44]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5843
           1       0.99      0.99      0.99      5362

    accuracy                           0.99     11205
   macro avg       0.99      0.99      0.99     11205
weighted avg       0.99      0.99      0.99     11205



In [45]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)

    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction: {}".format(
        output_lable(pred_LR[0]),
        output_lable(pred_DT[0]),
        output_lable(pred_RFC[0])
    ))


In [46]:
news = str(input())
manual_testing(news)

BRUSSELS (Reuters) - NATO allies on Tuesday welcomed President Donald Trump s decision to commit more forces to Afghanistan, as part of a new U.S. strategy he said would require more troops and funding from America s partners. Having run for the White House last year on a pledge to withdraw swiftly from Afghanistan, Trump reversed course on Monday and promised a stepped-up military campaign against  Taliban insurgents, saying:  Our troops will fight to win .  U.S. officials said he had signed off on plans to send about 4,000 more U.S. troops to add to the roughly 8,400 now deployed in Afghanistan. But his speech did not define benchmarks for successfully ending the war that began with the U.S.-led invasion of Afghanistan in 2001, and which he acknowledged had required an   extraordinary sacrifice of blood and treasure .  We will ask our NATO allies and global partners to support our new strategy, with additional troops and funding increases in line with our own. We are confident they w

In [47]:
news = str(input())
manual_testing(news)

Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, a

In [48]:
news = str(input())
manual_testing(news)

“Operation Sindoor has lent a new confidence and enthusiasm to the fight against terrorism across the world,” he added.  The prime minister also credited India's home-grown defence capabilities for the mission's success, following the spirit of 'Aatmanirbhar Bharat'.  Also Read | Maoism, Yoga Day, Operation Sindoor: What PM Modi talked about in ‘Mann Ki Baat’  “Our soldiers destroyed terror bases; it was their indomitable courage, along with the power of weapons, equipment and technology made in India. It also included the resolve of ‘Atmanirbhar Bharat’. The sweat of our engineers, our technicians, in fact, that of everyone… is involved in this victory.,” the prime minister said.  Modi also lauded the Tiranga Yatras organised in cities, villages and small towns of the country as a mark to honour the armed forces.  “Thousands of people came out holding the tricolour, to pay their respects & honour to the country's armed forces,” Modi told the nation through his monthly radio programme.